0# Подгружение нужных ресурсов для работы библиотеки (необязательно если уже есть)

In [ ]:
import fasttext.util
import nltk
print('Установка модели...')
fasttext.util.download_model('ru', if_exists='ignore')
try: 
    nltk.download('stopwords')
except:
    pass
print("Успешно!")

1# Получение самих отзывов с площадок
Мы использовали requests(запросы) для получения страницы сайта с отзывами, затем используя BeautifulSoup получая сами отзывы, очищая их от html-развертки

def get_review_from_bankiru - функция для получения отзывов с сайта банки.ру
def get_review_from_sravniru - функция для получения отзывов с сравни.ру
def save_reviews - функция для сохранения отзывов в json-файл

In [ ]:
import json
import requests
from bs4 import BeautifulSoup


def get_review_from_bankiru(num=100, page=1):
    revs = {"good": [], "bad": []}
    reviews = 0
    links = []
    while True:
        req = requests.get("https://www.banki.ru/services/responses/bank/tcs/" + f"?page={page}")
        soup = BeautifulSoup(req.content, "html.parser")
        for h3 in soup.find_all("h3", class_="text-weight-medium text-size-3 ldecc766d"):
            a = h3.find_all("a")
            links.append(a[0]['href'])
        for link in links:
            if reviews >= num:
                with open("reviews.json", "w") as f:
                    json.dump(revs, f)
                return revs
            req = requests.get("https://www.banki.ru" + link)
            soup = BeautifulSoup(req.content, "html.parser")
            for div in soup.find_all("div", class_="lb1789875 markdown-inside markdown-inside--list-type_circle-fill"):
                p = div.find_all("p")
                review = ""
                for r in p:
                    review += str(r.text).replace("\n", "").replace("\r", "").replace("\t", "")
                if review == "":
                    review += div.text.replace("\n", "").replace("\r", "").replace("\t", "")
                if review != "" and review != " " and review != "\n" and review != ', ':
                    if int(soup.find_all("div", class_="lbb810226")[0].find_all("div")[1].text) in (1, 2, 3):
                        revs["bad"].append(review)
                    elif int(soup.find_all("div", class_="lbb810226")[0].find_all("div")[1].text) in (4, 5):
                        revs["good"].append(review)
                        
                    reviews += 1
                    print(f"Получено отзывов с банки.ру: {reviews}/{num}")

        page += 1

def get_review_from_sravniru(num=100, page=1):
    revs = {"good": [], "bad": []}
    r = 0
    while True:
        if r >= num:
            with open("reviews.json", "w") as f:
                json.dump(revs, f)
            return revs
        req = requests.get(f"https://www.sravni.ru/proxy-reviews/reviews/?filterBy=withRates&fingerPrint=ea060f38d490a841e5bae143a1505423&isClient=true&locationRoute=&newIds=true&orderBy=byDate&pageIndex={page}&pageSize=10&reviewObjectId=5bb4f769245bc22a520a6353&reviewObjectType=banks&specificProductId=&withVotes=true")
        reviews = req.json()
        if reviews["items"] == []:
            
            return revs
        for item in reviews["items"]:
            review = item["text"].replace("\n", "").replace("\r", "").replace("\t", "")
            if review != "" and review != " " and review != "\n" and review != ', ':
                if int(item["rating"]) in (1, 2, 3):
                    revs["bad"].append(review)
                elif int(item["rating"]) in (4, 5):
                    revs["good"].append(review)
                    
                r += 1
                print(f"Получено отзывов с сравни.ру: {r}/{num}")
        page += 1
def save_reviews(*args: dict):
    revs = {"good": [], "bad": []}
    for review in args:
        revs["good"] += review["good"]
        revs["bad"] += review["bad"]
    with open("reviews.json", "w") as f:
        json.dump(revs, f) 

revs_s = get_review_from_sravniru(100, 1)
revs_b = get_review_from_bankiru(100, 1)
save_reviews(revs_s, revs_b)
print('Успешно!\n')

2# Очистка лишних данных в отзывах и приведение в нижний регистр
В данном фрагменте кода мы делали пред-обработку отзывов используя re для удаления от лишних символов(unicode-символы, буквы латинского алфавита), а так же удаляли различные стоп-слова с помощью библиотеки обработки естественного языка(nltk)

def delete_symbols - функция для удаления лишних символов в списке отзывов
def delete_stop_words - функция для удаления стоп-слов и привод в нижний регистр предложения(текста)
def lower_review - функция для удаления стоп-слов из списка отзывов (использует def delete_stop_words)
def save_lower_reviews - функция для сохранения результата(списка обновленных отзывов) в json-файл

In [ ]:
import re
import json
from nltk.corpus import stopwords

def delete_symbols(reviews_a):
    reviews_symbols = []
    print('Очистка лишних символов в отзывах...')
    for word in reviews_a:
        reg_sumbols = re.sub('[^\w\s]+|[\d]+', ' ', word)
        reg_eng_symbols = re.sub('[a-zA-Z\s]+', ' ', reg_sumbols)
        reviews_symbols.append(reg_eng_symbols.split())
    return reviews_symbols

def delete_stop_words(review_a): 
    
    stop_words = list(stopwords.words('russian'))
    reviews_stop_words = []
    for s in review_a:
        word = s.lower()
        if word not in stop_words and word != "\n":
            reviews_stop_words.append(word)
    return reviews_stop_words

def lower_review(reviews_a):
    print('Удаление стоп-слов из отзывов...')
    lower_reviews = []
    for rev in reviews_a:
        deleted_stop_words = delete_stop_words(rev)
        lower_reviews.append(deleted_stop_words)
    return lower_reviews

def save_lower_reviews(reviews_a):
    reviews_a = {"reviews": reviews_a}
    print('Сохранение отзывов...')
    with open('lower_reviews.json', 'w') as f:
        json.dump(reviews_a, f)

with open("reviews.json", "r", encoding='utf-8') as read_file:
    reviews_sorted = json.loads(read_file.read())
reviews = reviews_sorted["good"] + reviews_sorted["bad"]

reviews_deleted_symbols = delete_symbols(reviews)
reviews_lower = lower_review(reviews_deleted_symbols)
save_lower_reviews(reviews_lower)
del reviews_deleted_symbols, reviews_lower, reviews_sorted
print('Успешно!\n')

3# Преобразование отзывов в вектора
В предложенном фрагменте кода мы используем модель fasttext(а именно "cc.ru.300.bin") для преобразования отзывов в вектора, которые как раз машина сможет обрабатывать

def get_vector - функция для получения вектора из предложения(текста)
def to_vector - функция для преобразования списка отзывов в вектора с разделением на положительные и отрицательные(использует def get_vector)
def save_vector - сохраняет полученные вектора в json-файл

In [ ]:
import json

print("Загрузка модели...")
import fasttext.util
ft = fasttext.load_model('cc.ru.300.bin')


def get_vector(review):
    sentence = "".join(s + " " for s in review)
    sentence_vector = ft.get_sentence_vector(sentence)
    return sentence_vector

def to_vector(lower_reviews_a):
    print("Перевод отзывов в вектора...")
    reviews_v = {"good": [], "bad": []}
    for i in range(len(json.loads(open('reviews.json').read())["good"])):
        reviews_v["good"].append(get_vector(lower_reviews_a[i]).tolist())
    for i in range(len(json.loads(open('reviews.json').read())["bad"])):
        reviews_v["bad"].append(get_vector(lower_reviews_a[i + len(json.loads(open('reviews.json').read())["good"])]).tolist())
    return reviews_v
    

def save_vector(reviews_v):
    print("Сохранение векторов...")
    with open('vectorized_reviews.json', 'w') as f:
        json.dump(reviews_v, f)

with open("lower_reviews.json", "r", encoding='utf-8') as read_file:
    reviews_lower = json.loads(read_file.read())
reviews_vector = to_vector(reviews_lower["reviews"])
save_vector(reviews_vector)
print('Успешно!\n')

4# Суммаризация отзывов 
Здесь происходит удаление похожих на себя по содержанию отзывов. Мы используем библеотеку scipy.spatial для создания KD-дерева(дерево векторов). Используя наше дерево мы находим несколько наиболее схожих векторов на другие вектора, удаляя их и оставляя наиболее похожий(ближний). А в конце вывод полученных результатов

In [ ]:
from scipy import spatial
import numpy as np
import json

with open("vectorized_reviews.json", "r") as read_file:
    load_data = json.load(read_file)

vec_reviews_good = np.array(load_data["good"])
vec_reviews_bad = np.array(load_data["bad"])
print("Ищу общие схожести в отзывах...")
reviews = json.loads(open('reviews.json').read())
reviews_final = {"good": [], "bad": []}
for vec in vec_reviews_good:
    try:
        tree = spatial.KDTree(vec_reviews_good)
        similar_review = tree.query(vec, max(5, int(round(len(json.loads(open('reviews.json').read())["good"]) * 0.5) ** 0.5)))
        similar_vec = [vec_reviews_good[i].tolist() for i in similar_review[1]]
        for s in similar_vec:
            ind = load_data["good"].index(s)
    
            reviews["good"].pop(ind)
            r = reviews["good"][ind]
        reviews_final["good"].append(r)

    except:
        continue
r = None
for vec in vec_reviews_bad:
    try:
        tree = spatial.KDTree(vec_reviews_bad)
        similar_review = tree.query(vec, max(5, int(round(len(json.loads(open('reviews.json').read())["bad"]) * 0.5) ** 0.5)))
        similar_vec = [vec_reviews_bad[i].tolist() for i in similar_review[1]]
        for s in similar_vec:
            ind = load_data["bad"].index(s)
    
            reviews["bad"].pop(ind)
            r = reviews["bad"][ind]
        reviews_final["bad"].append(r)
    except:
        continue
print("\nПОЛОЖИТЕЛЬНЫЕ ЧЕРТЫ:\n")
print("\n\n".join(reviews_final["good"]))
print(len(reviews_final["good"]), len(json.loads(open('reviews.json').read())["good"]))
print("\nНЕГАТИВНЫЕ ЧЕРТЫ:\n")
print("\n\n".join(reviews_final["bad"]))
print(len(reviews_final["bad"]))
print(len(reviews_final["good"] + reviews_final["bad"]), len(json.loads(open('reviews.json').read())["good"] + json.loads(open('reviews.json').read())["bad"]))